## Importing Libraries

In [28]:
import os
import argparse
import logging
import yaml
from string import Template
import json
import random


import kfp
from kfp import dsl
from kfp.components import func_to_container_op

## Define Kubeflow Pipeline

In [ ]:
@func_to_container_op
def show_results(accuracy :float) -> None:
    print(f"Accuracy of neural network: {accuracy}")

@dsl.pipeline(name='demo_pipeline', description='Fits a simple neural network')
def first_pipeline():

    download = kfp.components.load_component_from_file('download_data/download_data.yaml')
    preprocess_data = kfp.components.load_component_from_file('preprocess_data/preprocess_data.yaml')
    train_model = kfp.components.load_component_from_file('train_model/train_model.yaml')

    download_task = download()
    preprocess_data_task = preprocess_data(download_task.output)
    train_model_task = train_model(preprocess_data_task.output)
    
    show_results(train_model_task.outputs['Accuracy'])
    
if __name__ == '__main__':
    file_name = 'pipeline.yaml'
    kfp.compiler.Compiler().compile(first_pipeline, file_name)

## Upload pipeline

In [96]:
import kfp
client=kfp.Client()
filepath = '/home/jovyan/AI-platform/kubeflow/example/' + file_name
name = 'kubeflow_demo_' + str(random.randint(30000,90000))
print("Uploaded pipeline:", name)
pipeline = client.pipeline_uploads.upload_pipeline(filepath, name=name)